# Identifying Users With Similar Buying Habits and Preferences

## 1. Data Preprocessing

### Load data

In [4]:
import pandas as pd

aisles_df = pd.read_csv('aisles.csv')
dept_df = pd.read_csv('departments.csv')
prodorder_prior_df = pd.read_csv('order_products__prior.csv')
productorder_train_df = pd.read_csv('order_products__train.csv')
order_df = pd.read_csv('orders.csv')
product_df = pd.read_csv('products.csv')


### Merge into one dataframe 

Only keep the users that exist in both 'prior' table and train eval set of 'orders' table.

In [5]:
# orders in prior merged with product names
prodname_order_prior = pd.merge(prodorder_prior_df, 
                                    product_df, how='left', on='product_id')
# Prior orders with user_id, product_id, product_name
userorder_prior_prod = pd.merge(prodname_order_prior, 
                                    order_df, how='left', on='order_id')

In [ ]:
# keep only those that same user_id exists in both train and prior
userorder_prior_prod_inner = pd.merge(userorder_prior_prod,
                                          order_df[order_df['eval_set']=='train'][['user_id','eval_set']], 
                                          how='inner', on='user_id')

In [ ]:
userorder_prior_prod_inner.head()

## 2. Feature Extraction

The features that will be extracted directly are:
* Mean of order_dow (order placed day of week)
* Mean of order_hour_of_day
* Mean of days_since_prior_order
* Total number of orders made
* Total number of products bought

Then we need another vectorized feature of product name: combine all the products name into one row per user, for word2Vector analysis.

In [ ]:
habits_user = userorder_prior_prod_inner[['user_id','order_id',
                                       'product_name','order_dow',
                                       'order_hour_of_day','days_since_prior_order']]

In [ ]:
import numpy as np
# Create a dataframe: average value of each user
user_average = habits_user.groupby('user_id')['order_dow',
                                'order_hour_of_day',
                                'days_since_prior_order'].agg(np.nanmean)
# Total number of orders of each user
user_order = habits_user.groupby('user_id').order_id.nunique()
user_average['num_of_orders'] = user_order
# Total number of products of each user
prod_num = habits_user.groupby('user_id')['order_id'].agg('count')
user_average['num_of_products'] = prod_num

In [ ]:
# add product name to each user
list_of_names = []
for p_name in habits_user.groupby('user_id')['product_name']:
        list_of_names.append(' '.join(p_name[1]))

# add the names to dataframe
user_average['product_name'] = list_of_names       

In [ ]:
# Take a glimpse of the dataset
user_average.head()

#userorder_prior_prod_inner.to_csv("../output/userorder_prior_prod_inner.csv")
#user_average.to_csv('../output/user_average.csv')

### Extract Vectorized Text Feature: Use PySpark word2Vector

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import word2Vector

spark = SparkSession.builder.appName("User Habit").getOrCreate()

# load data
# user_average = pd.read_csv("../output/user_average.csv")
prodname_df = pd.DataFrame(user_average['product_name'])
# product_doc_df = spark.createDataFrame(prodname_df)

In [ ]:
prodname_df.head()

In [ ]:
# Sample a fraction of data due to large computation
fraction_sample = 0.2
productname_sample_df  = prodname_df.sample(frac = fraction_sample, random_state=321)
userid_sample  = productname_sample_df.index
print(userid_sample)

In [ ]:
df_list = []
for row in productname_sample_df['product_name']:
    tuple = (row.split(' '),)
    df_list.append(tuple)

# Check by the length of output
print(len(df_list))

In [ ]:
# Input data of word2Vector: Each row is a bag of words from a sentence or document.
N = len(df_list)//100
mod = len(df_list) % 100
doc_df = spark.createDataFrame(df_list[0:100], ["product_name"])

for i in range(1,N):
    doc_df_sub = spark.createDataFrame(df_list[100*i:100*(i+1)], ["product_name"])
    doc_df = doc_df.union(doc_df_sub)
    
doc_df_sub = spark.createDataFrame(df_list[100*N:len(df_list)], ["product_name"])
doc_df = doc_df.union(doc_df_sub)

In [ ]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=5, minCount=0, inputCol="product_name", outputCol="res")
mdl = word2Vec.fit(doc_df)

res = mdl.transform(doc_df)

In [ ]:
# Vectors: densevector
features_vectored = [ ]
for row in res.collect():
    text, vector = row
    features_vectored.append(vector)

In [ ]:
# Get values from densevector into array
features_vectored_array=[]
for vectors in features_vectored:
    features_vectored_array.append(vectors.values)

In [ ]:
# Names of vectorized feature columns
column_names = []
for i in range(1,6):
    name = "vectorized_feature_" + str(i)
    column_names.append(name)
    
# A dataframe: each vectorized feature as one column
features_vectored_df = pd.DataFrame(np.array(features_vectored_array).reshape(len(df_list),5), 
                 columns = column_names)

# Add "user_id" column
features_vectored_df['user_id'] = userid_sample

In [ ]:
# Take a glimpse
features_vectored_df.head()

### Combine All Features: Concatenate word2Vector feature with other features into one dataframe

In [ ]:
# slice user_average with sampled user_id
sample_useravg = user_average[user_average.index.isin(userid_sample)]
# set index as one column 'user_id'
sample_useravg.reset_index(level=0, inplace=True)
# merge two dfs on 'user_id'
userfeatures_habits = pd.merge(sample_useravg, features_vectored_df, how='inner', on="user_id")
# drop "product_name"
userfeatures_habits.drop('product_name', axis=1, inplace=True)

In [ ]:
# Take a glimpse
userfeatures_habits.head()

In [ ]:
#userfeatures_habits.to_csv('../output/userfeatures_habits.txt', sep='\t', index=False)
#userfeatures_habits.to_csv('../output/userfeatures_habits.csv', sep='\t', index=False)
#userfeatures_habits_only = userfeatures_habits.loc[:, 'order_dow':]
#userfeatures_habits_only.to_csv('../output/userfeatures_habits_only.txt', sep='\t', index=False)
#userfeatures_habits_only.to_csv('../output/userfeatures_habits_only.csv', index=False)

## 3. Cluster Users: PySpark K-Means 

#### PCA: Reduce features to 2-dimensional

In [ ]:
from sklearn.decomposition import PCA
userfeatures_habits_only = pd.read_csv('../output/userfeatures_habits_only.csv')
userfeatures_habits = pd.read_csv('../output/userfeatures_habits.csv', sep = '\t')
pca = PCA(n_components=2).fit(userfeatures_habits_only)
pca_2d = pca.transform(userfeatures_habits_only)

In [ ]:
pca_dataframe = pd.DataFrame(pca_2d)
#pca_dataframe.to_csv('../output/pca_feature_df.txt', sep='\t', index=False)
#pca_dataframe.to_csv('../output/pca_feature_df.csv', index=False)
pca_dataframe['user_id'] = userfeatures_habits['user_id']
#pca_dataframe.to_csv('../output/pca_dataframe.txt', sep='\t', index=False)
#pca_dataframe.to_csv('../output/pca_dataframe.csv', index=False)

#### Find the optimal K

Find the optimal number of clusters by calculating the within set sum of squared error (WSSSE). As the number of cluster increases, WSSSE will decrease. The best choice is at the elbow of WSSSE graph.

In [ ]:
# Load and parse the data
datapca = sc.textFile("../output/pca_feature_df.txt")
parseddatapca = datapca.map(lambda line: array([float(x) for x in line.split('\t')]))

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [ ]:
WSSSE_listpca = []

K_range = range(5,185,5)
for K in K_range:
    
    # Build the mdl (cluster the data)
    clusters = KMeans.train(parseddatapca, K, maxIterations=10, initializationMode="random")
    
    WSSSE_pca = parseddatapca.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    print("====== k:"+str(K)+" -- Within Set Sum of Squared Error = " + str(WSSSE_pca) + "=======")
    WSSSE_listpca.append(WSSSE_pca)

In [ ]:
WSSSE_datapca = {'K':K_range, "WSSSE": WSSSE_listpca}
WSSSE_pca_dataframe = pd.DataFrame(WSSSE_datapca)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
WSSSE_pca_dataframe.plot(x='K', y='WSSSE')
plt.axvline(40, 
            color='darkorange', linestyle='dashed', linewidth=2)
plt.xlabel('Clusters')
plt.title('Within Set Sum of Squared Error of K-Means')
plt.show()
#fig.set_dpi(200)
#fig.savefig("../figs/WSSSE_pca.png")

The optimal k is usually one where there is an “elbow” in the WSSSE graph. So choose k = 40.

#### Run K-Means mdl with optimal K=40

In [ ]:
k_optimal = 40
clusters = KMeans.train(parseddatapca, k_optimal, maxIterations=10, initializationMode="random")
#clusters.save(sc, "../output/KMeansmdl_pca")
#clusters = KMeansmdl.load(sc, "../output/KMeansmdl_pca")

#### Get the cluster labels

In [ ]:
predict_clusters = clusters.predict(parseddatapca)
# Into a list
cluster_res = [ ]
for row in predict_clusters.collect():
    cluster_res.append(row)

#### Get the centers for each user

In [ ]:
def GetCenter(point):
    center = clusters.centers[clusters.predict(point)]
    return center

RDDCenter = parseddatapca.map(lambda point: GetCenter(point))

ress_center = [ ]
for row in RDDCenter.collect():
    ress_center.append(row)
    
ress_center = pd.DataFrame(ress_center,columns=['x','y'])

#### KMeans ress Summary

In [ ]:
summary_kmeans = ress_center
summary_kmeans['clusters'] = cluster_res
summary_kmeans['user_id'] = userfeatures_habits['user_id']

summary_kmeans = pd.merge(pca_dataframe, summary_kmeans ,how='inner', on='user_id')
#summary_kmeans.to_csv("../output/summary_kmeans.csv", header=True)

# x & y are coordinates of cluster centers
# 0 & 1 are coordinates of each user
summary_kmeans.head()

#### Visualization of Kmeans ress

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure()

# Unique category labels for clusters
labels_color = summary_kmeans['clusters'].unique()

# List of RGB triplets
rgb_values = sns.color_palette("Set2", 40)

# Map label to RGB
map_color = dict(zip(labels_color, rgb_values))

# Finally use the mapped values
plt.scatter(summary_kmeans['x'], summary_kmeans['y'], c=summary_kmeans['clusters'].map(map_color))
plt.title("Centers for K-Means Clusters")
plt.show()

#fig.set_dpi(200)
#fig.savefig('../figs/cluster_centers.png')

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

fig = plt.figure()

# Unique category labels for clusters
labels_color = summary_kmeans['clusters'].unique()

# List of RGB triplets
rgb_values = sns.color_palette("Set2", 40)

# Map label to RGB
map_color = dict(zip(labels_color, rgb_values))


# Finally use the mapped values
plt.scatter(summary_kmeans[0], summary_kmeans[1], c=summary_kmeans['clusters'].map(map_color), s = 0.5)
plt.title("K-Means Clusters")
plt.show()

#fig.set_dpi(300)
#fig.savefig('../figs/clusters.png')

#### Most Popular Products in Each User Cluster

In [ ]:
import pandas as pd
summary_kmeans = pd.read_csv("../output/summary_kmeans.csv")

# merge to get clusters corresponds to product_name
cluster_order_info = pd.merge(summary_kmeans, userorder_prior_prod_inner, how='left', on='user_id')
prod_cluster = cluster_order_info[['user_id','clusters','product_name']]

In [ ]:
# count the frequency of products in each cluster
count_cluster = prod_cluster.groupby(['clusters','product_name']).agg('count')
# reset indexes (twice)
# count_cluster.reset_index(level=0, inplace=True)
# count_cluster.reset_index(level=0, inplace=True)
# count_cluster.sort_values(['clusters','user_id'], ascending=False).groupby('clusters').head(5)

# Top 10 products in each cluster
top_prods = count_cluster['user_id'].groupby(level=0, group_keys=False).nlargest(10).reset_index()

In [ ]:
import matplotlib.pyplot as plt
#top_prods.columns.values[2]='count'
top_prods[top_prods['clusters'] == 0][['product_name','user_id']]

In [ ]:
widetop_products =top_prods.pivot(index='clusters', columns='product_name', values='user_id').fillna(0)
widetop_products_percent = widetop_products.div(widetop_products.sum(axis=0), axis=1)
longtop_products = widetop_products_percent.unstack().reset_index()
longtop_products.columns.values[2]='count'

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4,figsize=(20,40),sharey=True)
#plt.figure(figsize=(25,10))
#ax.spines['top'].set_visible(False)
#ax.spines['right'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
#ax.spines['left'].set_visible(False)
#ax.patch.set_visible(False)
#ax.grid(False)

grp1 = []
for i in range(10):
    grp1.append(i)
ax1.plot(widetop_products_percent.loc[grp1].transpose())
ax1.legend(widetop_products_percent.transpose().columns[0:10],title="Cluster ID",loc='upper right',prop={'size': 12})
ax1.set_title('Percent of Products in Each Cluster',size=20)
#ax1.set_xticklabels(rotation=90, size=12)

grp2 = []
for i in range(10,20):
    grp2.append(i)
ax2.plot(widetop_products_percent.loc[grp2].transpose())
ax2.legend(widetop_products_percent.transpose().columns[10:20], title="Cluster ID",loc='upper right',prop={'size': 12})
#ax2.set_xticklabels(rotation=90, size=12)

grp3 = []
for i in range(20,30):
    grp3.append(i)
ax3.plot(widetop_products_percent.loc[grp3].transpose())
ax3.legend(widetop_products_percent.transpose().columns[20:30],title="Cluster ID",loc='upper right',prop={'size': 12})
#ax3.set_xticklabels(rotation=90, size=12)

grp4 = []
for i in range(30,40):
    grp4.append(i)
ax4.plot(widetop_products_percent.loc[grp4].transpose())
ax4.legend(widetop_products_percent.transpose().columns[30:40],title="Cluster ID",loc='upper right',prop={'size': 12})
#ax4.set_xticklabels(rotation=90, size=12)
#plt.xticks(rotation=90, size=12)

for ax in fig.axes:
    plt.sca(ax)
    plt.xticks(rotation=90, size=12)
    
plt.subplots_adjust(wspace=0, hspace=0.7)
#fig.tight_layout()

plt.show()
fig.set_dpi(300)
fig.savefig('../figs/prod_cluster_frequency.png')